<h1>Drug Use Data Cleaning Project </h1>


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
data = pd.read_csv('35509-0001-Data.tsv',sep='\t')

In [6]:
data.head()

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,99,99,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,99,99,14,9999,99,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,99,99,14,9999,99,1,2,...,1,99,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,99,99,16,9999,99,4,93,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,99,99,14,9999,99,4,93,...,1,1,1,1,2,2,2,5651.73,30013,2


<h1> Part 1) Removing columns where over 25% of answers are skipped, left blank, or refused to answer </h1>

<p> <b>Justification</b>: for purposes of analysis, a question where over 25% of survey takers didn't answer is less valuable to analyze and make predictions with. Questions with large amounts of missing answers can also bias analysis as it is unclear why such questions are skipped. </p>
<p><b>Note</b>: the dataset contains no 'NaN' values or other markers that may be used to indicate missing or faulty data other than the following key codes below </p>


97,997,9997,and 99997 = Refused <br>
98, 998, 9998, 99998 = Blank (No Answer) <br>
99, 999, 9999, 99999 = Legitamate Skip <br>
-9.00 = Missing



Goal for percent_missing function: For each column add the percentages of each missing value key-code together. Return the total percentage of all missing value key-codes. 

In [7]:
def percent_missing(series):
    vc = series.value_counts(normalize=True)
    total_percent = 0 
    try:
        total_percent += vc[97]
    except:
        total_percent += 0 
    try:
        total_percent += vc[98]
    except:
        total_percent +=0
    try:
        total_percent += vc[99]
    except:
        total_percent +=0
    try:
        total_percent += vc[997]
    except:
        total_percent += 0
    try:
        total_percent += vc[998]
    except:
        total_percent += 0
    try:
        total_percent += vc[999]
    except:
        total_percent += 0 
    try:
        total_percent += vc[9997]
    except:
        total_percent +=0
    try:
        total_percent += vc[9998]
    except:
        total_percent += 0
    try:
        total_percent += vc[9999]
    except:
        total_percent += 0 
    #For now
    try:
        total_percent += vc[99997]
    except:
        total_percent += 0
    try:
        total_percent += vc[99998]
    except:
        total_percent += 0
    try:
        total_percent += vc[99999]
    except:
        total_percent += 0 
    try:
        total_percent += vc[-9.00]
    except:
        total_percent += 0     
    
    return total_percent
    

In [8]:
data['DR5DAY'].value_counts(normalize=True).sort_index(ascending=False).head()

98    0.012527
97    0.001414
94    0.003626
93    0.249094
91    0.308285
Name: DR5DAY, dtype: float64

In [9]:
data['DR5DAY'].value_counts(normalize=True)[97] + data['DR5DAY'].value_counts(normalize=True)[98]

0.013941261783901378

In [10]:
percent_missing(data['DR5DAY'])

0.013941261783901378

<h5> Dropping all columns with over 25% reported answers being a skip, left blank, or refused to answer </h5>

In [11]:
drop_cols = []
for i in data.columns:
    if percent_missing(data[i]) > 0.25:
        drop_cols.append(i)

<i> Appears to be 1299 columns of this kind, so 1299 columns can be dropped based on this criteria </i>

In [12]:
len(drop_cols)

1299

In [13]:
data_dropped = data.drop(drop_cols,axis=1)
data_dropped.head()

,CASEID,QUESTID2,CIGEVER,CIGTRY,CIGREC,CIG30USE,CG30EST,CIG30AV,CIG30BR2,CIG30TPE,...,IIEMPSTY,II2EMSTY,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,19,1,1,99,2,9994,98,...,1,1,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,14,2,93,93,93,9993,93,...,1,1,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,14,1,2,99,1,112,4,...,1,1,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,16,4,93,93,93,9993,93,...,1,1,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,14,4,93,93,93,9993,93,...,1,1,1,1,2,2,2,5651.73,30013,2


<h1> Part 2) Dropping all columns containing less than 30 unique possible values </h1>
<p> <b> Justification </b>: Most of the most valuable columns for purposes of predictions are numeric columns. This is because numeric columns in general tend to be better for predictions, but also because the information of a significant number of other non-numeric columns is already contained in numeric columns. For example, columns of the form "have you used X in the past 30 days: yes or no" have such information implicit in columns of the form "number of days where used X in the past month (with 0 days indicating a no and any value 1 or greater indicating a yes". The cutoff of 30 for determining what constitutes a row worth dropping was chosen as most useful numeric data is in regards to measuring activity within a month, which has over 30 unique possible values. </p>

In [14]:
short_cols = []
for i in data_dropped.columns:
    if len(data_dropped[i].value_counts()) < 30:
        short_cols.append(i)

In [15]:
len(short_cols)

1690

In [16]:
data_dropped_2 = data_dropped.drop(short_cols,axis=1)
data_dropped_2.head()

,CASEID,QUESTID2,CIGTRY,CIG30USE,CIG30BR2,CIGAGE,SNUFTRY,SNF30USE,CHEWTRY,CHW30USE,...,TRNAGLST,STMAGLST,MTHAGLST,SEDAGLST,NMERTMT2,WTPOUND2,BMI2,CADRLAST,ANALWT_C,VESTR
0,1,48694667,19,1,9994,999,991,91,991,91,...,991,991,991,991,0,154,23.41,991,4398.40,30017
1,2,88530883,14,93,9993,999,16,15,991,91,...,17,991,991,991,1,165,21.77,15,1419.19,30052
2,3,33251077,14,2,112,999,991,91,15,93,...,991,991,991,991,0,130,18.65,993,14052.62,30028
3,4,37814127,16,93,9993,999,991,91,991,91,...,991,991,991,991,0,9997,-9.00,2,10848.18,30055
4,5,18762590,14,93,9993,999,991,91,991,91,...,991,991,991,991,0,150,23.49,1,5651.73,30013


<p> 152 columns remain </p>

<h1> Part 3) Manually evaluating relevance for purposes of analysis of remaining columns </h1>

In [17]:
#Dropping ID numbers which have no use for purposes of analysis
#final_data is to be the final cleaned dataset to be exported
final_data = data_dropped_2.drop(['CASEID','QUESTID2'],axis=1)

In [18]:
#Just printing the remaining columns
for i in final_data.columns.sort_values():
    print(i)

#AlCDAYS -- Good, Days drinken alcohol in past month (30 days)
#ALCTRY -- Good, Age of first using alcohol (includes response 991 meaning never drank alcohol, as do similar columns)
#ALCYRTOT -- Good, Days drinken alchohol in past year
#ANALAGE -- Good, Age when first used painkillers non-medically
##ANALNEWA -- BAD, specific name of pain killer used
##ANALNEWB -- BAD, specific name of 2nd pain killer used
##ANAlNEWC -- BAD, specific name of 3rd pain killer used
##ANALWT_C -- BAD, unclear what this variable is (and it doesn't seem useful based on its values)
##ANLAGLST -- Good for now, age when last used painkiller. Not sure if it'll be useful
##ANLYRTOT -- Good for now, days used painkiller in last year (not including non-persciption pain killers)
##BLNT30DY -- Bad, days used blunt in past 30 days, doubles data with both cigarettes and marijuana
##BLNTAGE -- Bad, ' '
##BMI2 -- Possibly good, BMI
##CADRLAST -- Possibly good, # of drinks at last drinking event
##CCDAYPYR -- Good, number of days used cocaine (in any form, i.e. Crack) in the past year
##CGR30BR2 -- Bad, specific cigarrete brand used (if any at all)
##CGR30USE -- Bad, # of days used cigars in last 30 days. 71% of people never used cigars, and seeing as questions about
## cigarettes are in this dataset too, it seems best to include all cigar columns (CGR)
##GGRAGLST -- Bad
##CHEWTRY -- Bad, Age when first used chewing tobacco. Bad for reasons the same as not including cigars.
##CHW30BR2 -- Bad
##CHW30USE -- Bad
##CHWAGLST -- Bad
##CIG30BR2 -- Bad, Specific cigarrete brand used if any
##CIG30USE -- Good, days used cigarretes in past 30 days
##CIGAGE -- Good, age when first started smoking cigarretes
##CIGAGLST -- Possibly Good, age last used cigarretes

##Can create a feature, years since last used cigarretes (CIGAGLST - Age)

##CIGARTRY -- Bad
##CIGTRY -- Possibly Good, Age when first smoked a cigarrete
##COCAGE -- Possibly Good, Age when first used cocaine
##COCAGLST -- Possibly Good, Age when last used cocaine
##COCYRTOT -- Good, # of days in last year used cocaine
##COLDYR1 -- Bad, name of specific cough medications used
##COLDYR2 -- Bad
##CRKAGE -- Bad, Age when last used crack. Bad since q's about cocaine included crack
##CRKAGLST -- Bad '   '
##CRKYRTOT -- Bad '   '
##DR5DAY -- Probably Bad, Days drank 5 or more alchoholic drinks in past 30 days. Has too much info from other columns in it.
##ECSAGE -- Possibly Good, Age when first used ecstacy
##ECSAGELST -- Possibly Good, age when last used ecstacy
##HALAGLST -- Possibly Good, age when last used a hallucinogen
##HALLAGE -- Possibly Good, Age when first used hallucinogen
##HALNEWA -- Bad, Name of specific hallucigen
##HALNEWB -- Bad ' '
##HALNEWC -- BAD ' '
##HALYRTOT -- Possibly Good, total days used hallugen in past year
##HERAGE -- Good, Age when first used Heroin
##HERAGLST -- Bad, since 98.5% of respondents never used heroin, HERAGE might not even be good, but I want it to have at 
# least one feature on heroin, and HERAGE is best
##HERYRTOT -- Bad ' '
##HRDAYPYR -- Bad ' '
##HLDAYPYR -- Possibly Good, Days used hallucanegens in past year (can be chunked into categories maybe)
##IEMAGE -- Good, Age first used any illicit drug other than marijuana
##INDAYPYR -- Possibly Good, Days used inhalents in past year
##INHAGE -- Possibly Good, age when first used inhalent
##INHAGLST -- Possibly Good, age when last used inhalent
##INHNEWA -- Bad, specific inhalent used
##INHYRTOT -- Probably bad, the mode is 1 and only 0.3 of people wrote 1. May be better to just take a binary used-notused
##IRALCAGE -- Good, Age first used alcohol
##IRALCD5 -- Possibly good, days drank alcohol in past month (may best to only have this or the yearly stat but not both)
##IRALCFM -- Bad, alcohol frequency past month (same data as IRALCD5 but in different form)
##IRALCFY -- Bad, same as ALCDAYS but different form
##IRANLAGE -- Possibly good, age of first use of painkillers
##IRANLFY -- Bad, same as IRANLAGE but different form
##IRCDUAGE --Bad, daily age of first cigarrete use (IRCIGAGE makes this category obsolete)
##IRCGAVD -- Bad, Nicotine dependence score (would require understanding the metrics of the test, plus most people scored)
## either a ('Never used cigarretes' or a 1.00, so the data would most likely not be too useful if at all anyway)
##IRCGLMR -- Bad, difficult to interpret test of previous dependence smoking
##IRCGRAGE -- Bad, age of first cigar use (I'm not using cigar data for reasons noted above)
##IRCGRFM -- Bad, cigar related
##IRCHWAGE -- Bad, chew related (not using chew data)
##IRCHWFM -- Bad, chew related
##IRCIGAGE -- Good, age of first cigarrete use
##IRCIGFM -- Bad, same as IRCIGAGE but different form
##IRCOCAGE -- Possibly good, imputation revised age of first use of cocaine

## NOTE on categories beginning with IR (imputation revised) They're the same or similar to their counterparts (will 
## explore further in next step of analysis)

##IRCOCFY -- Bad, cocaine use frequency in last year. I'm not using frequencies
##IRCRKAGE -- Bad, Age of first used crack. Not using crack since cocaine q's include crack
##IRCRKFY -- Bad
##IRECSAGE -- Possibly good, age first used ecstacy
##IRHALAGE -- Possibly good, age first used hallucinogen
##IRHALFY -- Bad, frequency of hallucinogen (not using frequencies)
##IRHERAGE -- Possibly good, age of first heroin use
##IRHERFY -- Bad, frequency data
##IRINHAGE -- Possibly good, age of first inhalent use
##IRINHFY -- Bad, inhalent frequency
##IRLSDAGE -- Bad, LSD first usg age. LSD is included in hallucinogen q's
##IRMJAGE -- Possibly good, Marijuana first time use
##IRMJFM -- Bad, Marijuana frequency last month
##IRMJFY -- Bad, Marijuana frequency last year
##IRMTHAGE -- Good, age first used methanphetamines (meth)
##IRMTHFY -- Bad, meth frequency last year
##IROXYAGE -- Bad, age first used oxycontin (bad since doubles-data with columns on pain-killers)
##IROXYFY -- Bad, oxycontin related (see above)
##IRPCPAGE -- Bad, agefirst used PCP. Doubles data with hallucinogens
##IRSEDAGE -- Probably bad, age first used sedative. May double data with other columns
##IRSEDFY -- Bad, sedative frequency
##IRSLTAGE -- Bad, first use of smokeless tobacco (may double-data with cigarrettes)
##IRSNFAGE -- Bad, first use of snuff (may double-data with cigarretes)
##IRSNFFM -- Bad, snuff frequency use 
##IRSTMAGE -- Possibly good, age of first using stimulants
##IRSTMFY -- Bad, stimulant freq per year
##IRTRNAGE -- Possibly good, age first used tranquilizer
##IRTRNFY -- Bad, tranquilizer frew per year
##LSDAGE -- Bad, age when first used LSD (LSD included in hallucinogens)
##LSDAGLST -- Bad, LSD related (not using lsd)
##METHAGE -- Possibly good, age when first used meth (not imputation revised)
##MJAGE -- Possibly good, age when first used marijuana (not imputation revised)
##MJDAY30A -- Good, days used marijuana in past 30 days
##MJYRTOT -- Good, days used marijuana in past year
##MMBT30DY -- Possibly good, days bought marijuana in past 30 days
##MMLSPCAT -- Bad, price category bought marijuana at (will double data with number of columns)
##MRDAYPMO -- Possibly good, ave number of days user marijuana per month in past year (will have to choose bw this and MJDAY30A)
##MRDAYPYR -- Possibly good, days user marijuana per year (will choose bw this and MJYRTOT)
##MRJAGLST -- Possibly good, age when last used marijuana 

## can subtract MRJAGLST from age for possibly interesting feature

##MTDAYPYR -- Possibly good, days used meth in last year
##MTHAAGE -- Possibly good, age when first used meth
##MTHAGLST -- Possibly good, age when last used meth

## can subtract MTHAGLST from age

##MTHAYTOT -- Possibly good, total number days used meth past year
##MTHYRTOT -- Possibly good, may be same as MTHAYTOT, will have to choose bw the two
##NMERTMT2 -- Good, times in emergency room in past month
##NODR30A -- Bad, usual number of drinks per day in past 30 days (doubles data with alcdays)
##OXDAYPYR -- Bad, oxycontin (doubles data with painkillers)
##OXYAGLST -- Bad, ' '
##OXYCAGE -- Bad, ' '
##OXYYRTOT -- Bad, ' '
##PCPAGE -- Bad, pcp (doubles data with hallucinogens)
##PCPAGLST -- Bad, ' '
##PRDAYPMO -- Good, days used painkillers per month (not perscribed or otc)
##PRDAYPYR -- Good, days used painkillers in year
##PSYAGE2 -- Possibly good, age of first use of psychotherapeutics

## NOTE: Psychotherapeutics includes sedatives, tranquilizers, stimulants, and analgesics. 
## may be best to drop those columns and keep the general PSYAGE2 column

##SEDAGE -- Possibly good, age when first used sedatives (non-medically)
##SEDAGLST -- Possibly good, age when last used sedatives
##SEDNEWA -- Bad, name of specific brand of sedative
##SEDYRTOT -- Possibly good, total number of days used sedatives in last year
##SLT30BR2 -- Bad, specificy name of smokeless tobacco
##SLTTRY -- Bad, Age when first tried smokeless tobacco (doubles data with cigarretes)
##SNF30USE -- Bad, days used snuff in past 30 days (doubles data with cigarretes)
##SNFAGLST -- Bad, snuff ' '
##SNUFTRY -- Bad, snuff ' '
##STDAYPYR -- Possibly good, days used stimulants in past year
##STIMAGE -- Possibly good, age when first used stimulants
##STIMNEWA -- Bad, specific stimulant used

#### Stimulants include meth (may have to check some things over)

##STMAGLST -- Possibly good, age last used stimulant
##STMYRTOT -- Possibly good, days used stimulants in past year
##SUMAGE -- Possibly good, age of first use of any illicit drug ##FLAGGED FOR CLOSER LOOK
##TRANAGE -- Possibly good, may double data with neurohallicnogens (check later)
##TRDAYPMO -- Possibly good, avg days per month used tranquelizers last year
##TRDAYPYR -- Possibly good, days used tranquellizers last year
##TRNAGLST -- Possibly good, age when last used tranquellizers
##TRNEWA -- Bad, name of specific tranquelizer used
##TRNYRTOT -- Possibly good, total days used tranquellizers last year (will decide bw this and TRDAYPYR)
##VESTR -- Bad, another id code
##WTPOUND2 -- Good, weight in pounds

good_cols = ['ALCDAYS','ALCTRY','ANALAGE','ANLAGLST','ANLYRTOT','BMI2','CADRLAST','CCDAYPYR',
            'CIG30USE','CIGAGE','CIGAGLST','CIGTRY','COCAGE','COCAGLST','COCYRTOT','ECSAGE','ECSAGLST','HALAGLST',
            'HALLAGE','HALYRTOT','HERAGE','HLDAYPYR','IEMAGE','INDAYPYR','INHAGE','INHAGLST','IRALCAGE','IRALCD5','IRCIGAGE',
            'IRCOCAGE','IRECSAGE','IRHALAGE','IRHERAGE','IRINHAGE','IRMJAGE','IRMTHAGE','IRSTMAGE','IRTRNAGE','METHAGE',
            'MJAGE','MJDAY30A','MJYRTOT','MMBT30DY','MRDAYPMO','MRDAYPYR','MRJAGLST','MTDAYPYR','MTHAAGE','MTHAGLST',
            'MTHAYTOT','MTHYRTOT','NMERTMT2','PRDAYPMO','PRDAYPYR','PSYAGE2','SEDAGE','SEDAGLST','SEDYRTOT',
            'STDAYPYR','STIMAGE','STMAGLST','STMYRTOT','SUMAGE','TRANAGE','TRDAYPMO','TRDAYPYR','TRNAGLST','TRNYRTOT',
            'WTPOUND2','ALCYRTOT','IRANLAGE']
bad_cols = ['ANALNEWA','ANALNEWB','ANALNEWC','ANALWT_C','CGR30BR2','CGR30USE','CGRAGLST','CHEWTRY','CHW30BR2','CHW30USE',
           'CHWAGLST','CIG30BR2','CIGARTRY','COLDYR2','COLDYR1','CRKAGE','CRKAGLST','CRKYRTOT','DR5DAY','HALNEWA',
           'HALNEWB','HALNEWC','HERAGLST','HERYRTOT','HRDAYPYR','INHNEWA','INHYRTOT','IRALCFM','IRALCFY','IRANLFY',
           'IRCGAVD','IRCGLMR','IRCGRAGE','IRCGRFM','IRCHWAGE','IRCHWFM','IRCDUAGE','IRCIGFM','IRCOCFY','IRCRKAGE',
           'IRCRKFY','IRHALFY','IRHERFY','IRINHFY','IRLSDAGE','IRMJFM','IRMJFY','IRMTHFY','IROXYAGE','IROXYFY',
           'IRPCPAGE','IRSEDAGE','IRSEDFY','IRSLTAGE','IRSNFAGE','IRSNFFM','IRSTMFY','IRTRNFY','LSDAGE','LSDAGLST',
           'MMLSPCAT','OXDAYPYR','OXYAGLST','OXYCAGE','OXYYRTOT','PCPAGE','PCPAGLST','SEDNEWA','SLT30BR2','SLTTRY',
           'SNF30USE','SNFAGLST','SNUFTRY','STIMNEWA','TRNEWA','VESTR','BLNT30DY','BLNTAGE','NODR30A']


ALCDAYS
ALCTRY
ALCYRTOT
ANALAGE
ANALNEWA
ANALNEWB
ANALNEWC
ANALWT_C
ANLAGLST
ANLYRTOT
BLNT30DY
BLNTAGE
BMI2
CADRLAST
CCDAYPYR
CGR30BR2
CGR30USE
CGRAGLST
CHEWTRY
CHW30BR2
CHW30USE
CHWAGLST
CIG30BR2
CIG30USE
CIGAGE
CIGAGLST
CIGARTRY
CIGTRY
COCAGE
COCAGLST
COCYRTOT
COLDYR1
COLDYR2
CRKAGE
CRKAGLST
CRKYRTOT
DR5DAY
ECSAGE
ECSAGLST
HALAGLST
HALLAGE
HALNEWA
HALNEWB
HALNEWC
HALYRTOT
HERAGE
HERAGLST
HERYRTOT
HLDAYPYR
HRDAYPYR
IEMAGE
INDAYPYR
INHAGE
INHAGLST
INHNEWA
INHYRTOT
IRALCAGE
IRALCD5
IRALCFM
IRALCFY
IRANLAGE
IRANLFY
IRCDUAGE
IRCGAVD
IRCGLMR
IRCGRAGE
IRCGRFM
IRCHWAGE
IRCHWFM
IRCIGAGE
IRCIGFM
IRCOCAGE
IRCOCFY
IRCRKAGE
IRCRKFY
IRECSAGE
IRHALAGE
IRHALFY
IRHERAGE
IRHERFY
IRINHAGE
IRINHFY
IRLSDAGE
IRMJAGE
IRMJFM
IRMJFY
IRMTHAGE
IRMTHFY
IROXYAGE
IROXYFY
IRPCPAGE
IRSEDAGE
IRSEDFY
IRSLTAGE
IRSNFAGE
IRSNFFM
IRSTMAGE
IRSTMFY
IRTRNAGE
IRTRNFY
LSDAGE
LSDAGLST
METHAGE
MJAGE
MJDAY30A
MJYRTOT
MMBT30DY
MMLSPCAT
MRDAYPMO
MRDAYPYR
MRJAGLST
MTDAYPYR
MTHAAGE
MTHAGLST
MTHAYTOT
MTHYRTOT
NMERTMT2
NODR30A
OXDAYPY

In [19]:
len(final_data.columns) - len(bad_cols) - len(good_cols)

0

<p> Stage one of manual analysis complete. Tests have been run to ensure that each column was imputed into either good_cols or bad_cols correctly, without typos or double dipping (that is, put into both good_cols and bad_cols). </p>

<p> Current category classes remaining </p>
<ul>
    <li>Alcohol</li>
    <li>Painkillers</li>
    <li>BMI</li>
    <li>Cocaine</li>
    <li>Cigarretes</li>
    <li>Ecstacy</li>
    <li>Hallucinogens</li>
    <li>Heroin</li>
    <li>Any Illicit drug not including marijuna</li>
    <li>Inhalents</li>
    <li>Pain killers</li>
    <li>Marijuana</li>
    <li>Tranquellizers</li>
    <li>Days bought marijuna </li>
    <li>Days in emergency room</li>
    <li>Psychotherapeutics</li> 
    <li>Sedatives</li>
    <li>Stimulants</li>
    <li>Any illiciut drug whatsoever</li>
    <li>Wheight in pounds</li>
</ul>

<h1>Part 4) Converting remaining columns to pure numeric data </h1>
<p>
    Part 5 will prune the data further by pruning columns which covary highly with other columns, as well as explore other possible means of pruning based on columns numeric relationships with each other (as well as a further sweep of manual analysis). Before any of this can be done, the remaining data must be purely numeric, which it currently is not. And so, part 4. </p>

In [20]:
pfd = final_data.drop(bad_cols,axis=1) ##pfd -- part four data

<p>First it is useful to isolate the numeric codes in each column which don't represent a true numeric value. The technique I'll use to do this is to first convert each instance of any number 1-30 to 0 in a temporary copy of the data-set. Numbers 1-30 always represent numerical data. After this step, I'll look at the unique values for each column. Since numbers 1-30 will all be 0, the only remaining unique values will be 0, and whatever's left, making whatevers left easy to identify </p>

In [21]:
mod_pfd = pfd
def one_to_thirty(element):
    if element >= 1 and element<=366:
        return 0
    else:
        return element
for i in mod_pfd.columns:
    mod_pfd[i] = mod_pfd[i].apply(one_to_thirty)
mod_pfd.head()

,CIGTRY,CIG30USE,CIGAGE,ALCTRY,ALCYRTOT,ALCDAYS,MJAGE,MJYRTOT,MRDAYPYR,MRDAYPMO,...,INHAGLST,ANLAGLST,TRNAGLST,STMAGLST,MTHAGLST,SEDAGLST,NMERTMT2,WTPOUND2,BMI2,CADRLAST
0,0,0,999,991,991,0,991,991,991,0,...,991,991,991,991,991,991,0,0,0.0,991
1,0,0,999,0,0,0,0,0,0,0,...,991,991,0,991,991,991,0,0,0.0,0
2,0,0,999,0,993,0,991,991,991,0,...,991,991,991,991,991,991,0,0,0.0,993
3,0,0,999,0,0,0,991,991,991,0,...,991,991,991,991,991,991,0,9997,-9.0,0
4,0,0,999,0,0,0,991,991,991,0,...,991,991,991,991,991,991,0,0,0.0,0


In [22]:
unique_codes = []
#shelving WTPOUND2 & BMI2 for now
for i in mod_pfd.columns.drop(['BMI2','WTPOUND2']):
    for v in mod_pfd[i].value_counts().index:
        if v not in unique_codes:
            unique_codes.append(v)
unique_codes.sort()
unique_codes.remove(0)
unique_codes

[981, 985, 989, 990, 991, 993, 994, 997, 998, 999]

<ol>
    <li> 981 -- Used in multiple columns. Represents "never used [column substance] logically assigned" </li>
    <li> 985 -- Used in multiple columns. Means "Bad data -- logically assigned". Occurs infrequently </li>
    <li> 989 -- Represents "legitamate skip -- logically assigned". For instances of skipping survey response contradictions</li>
    <li> 990 -- Represents "dont know if used [column substance]" </li>
    <li> 991 -- Used frequently, represents "Never used [column substance]" </li>
    <li> 993 -- Represents, "Used [column substance] but not in time period of question" </li>
    <li> 994 -- Represents "dont know"</li>
    <li> 997 -- Represents "refused" </li>
    <li> 998 -- Represents "blank"</li>
    <li> 999 -- Represents "legitamate skip", for cases where conditions of question don't apply</li>
</ol>


   
   

In [23]:
row_occurences = {}
for i in unique_codes:
    row_occurences[i] = mod_pfd[(mod_pfd == i).any(axis=1)].shape[0]
row_occurences

{981: 298,
 985: 3711,
 989: 150,
 990: 1,
 991: 55109,
 993: 27086,
 994: 2982,
 997: 1119,
 998: 2260,
 999: 25196}

<p> Solutions in this approach:
    For 981, 991, 993, 990: Will set to 0
    
    Remaining: 6
    985 -- bad data
    989 -- legitamate skip
    994 -- dont know
    997 -- refused
    998 -- blank
    999 -- legitamate skip
</p>

<p> Creating a new dataframe with 981,991, 993, and 994 values set to 0. ALso dropping WTPOUND2 and BMI2 from dataframe. I will potentially add these columns back in after analyzing them on their own. </p>

In [24]:
def ele_filter(element):
    if element == 981 or element == 991 or element == 993:
        return 0
    else:
        return element
mod_v2 = mod_pfd.applymap(ele_filter)
mod_v2 = mod_v2.drop(['WTPOUND2','BMI2'],axis=1)

In [25]:
mod_v2

,CIGTRY,CIG30USE,CIGAGE,ALCTRY,ALCYRTOT,ALCDAYS,MJAGE,MJYRTOT,MRDAYPYR,MRDAYPMO,...,HALAGLST,ECSAGLST,INHAGLST,ANLAGLST,TRNAGLST,STMAGLST,MTHAGLST,SEDAGLST,NMERTMT2,CADRLAST
0,0,0,999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,999,0,0,0,0,0,0,0,...,999,0,0,0,0,0,0,0,0,0
2,0,0,999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55156,0,0,999,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55158,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<p> Now checking to see if any columns are responsible for a significant portion of the problematic values <br>
Approach: use dataframe.value_counts(). Every item in the index other than 0 will be a problematic value</p>

In [26]:
total_rows = mod_v2.shape[0]
total_zeros_name = []
total_zeros_count = []
for i in mod_v2.columns:
    total_zeros_name.append(i)
    total_zeros_count.append(mod_v2[i].value_counts()[0])
zeros_dataframe = pd.DataFrame({'Name':total_zeros_name,'Count':total_zeros_count})

not_full = zeros_dataframe[zeros_dataframe['Count'] != 55160].sort_values(by='Count')
full_cols = zeros_dataframe[zeros_dataframe['Count'] == 55160]
print(not_full)
print('\n\n ----- \n\n')
print(full_cols)

        Name  Count
2     CIGAGE  41536
57  CIGAGLST  42412
56  MRJAGLST  48234
8   MRDAYPYR  48481
62  ANLAGLST  52908
23  PRDAYPYR  53179
53   MTHAAGE  53656
54  MTHAYTOT  53665
67  NMERTMT2  53705
59  HALAGLST  53949
63  TRNAGLST  54068
61  INHAGLST  54158
58  COCAGLST  54288
64  STMAGLST  54332
27  TRDAYPYR  54379
4   ALCYRTOT  54423
68  CADRLAST  54438
17  HLDAYPYR  54486
21   ANALAGE  54501
60  ECSAGLST  54503
31  STDAYPYR  54552
22  ANLYRTOT  54575
3     ALCTRY  54577
20  INDAYPYR  54649
13  CCDAYPYR  54684
7    MJYRTOT  54702
65  MTHAGLST  54776
0     CIGTRY  54795
19    INHAGE  54814
16  HALYRTOT  54851
66  SEDAGLST  54859
15   HALLAGE  54904
25   TRANAGE  54945
6      MJAGE  54945
26  TRNYRTOT  54946
34  MTDAYPYR  54960
29   STIMAGE  54968
30  STMYRTOT  54971
35    SEDAGE  54984
36  SEDYRTOT  55002
12  COCYRTOT  55051
18    ECSAGE  55073
32   METHAGE  55089
33  MTHYRTOT  55089
11    COCAGE  55096
14    HERAGE  55106


 ----- 


        Name  Count
1   CIG30USE  55160
5    ALC

<p> A quick takeaway here is that that the imputation revised columns are better seeing as they tend to be full. The code below will remove the non-imputation-revised counterparts present in the dataset </p>

In [27]:
full_cols[full_cols['Name'].str.contains('IR')]

,Name,Count
37,IRALCD5,55160
38,IRCIGAGE,55160
39,IRALCAGE,55160
40,IRMJAGE,55160
41,IRCOCAGE,55160
42,IRHERAGE,55160
43,IRHALAGE,55160
44,IRECSAGE,55160
45,IRINHAGE,55160
46,IRANLAGE,55160


In [28]:
counterparts = ['ALCDAYS','CIGAGE','ALCTRY','MJAGE','COCAGE','HERAGE','HALLAGE','INHAGE','ANALAGE','TRANAGE','STIMAGE','MTHAAGE','METHAGE']
##For some reason, there are 2 non imputation revied methage categories. I've removed both

In [29]:
not_full = not_full[not_full['Name'].isin(counterparts) == False].copy()
##Now there are 33 not_full categories
##A number of them contain "LST -- indicating they are of the form 'age when last used [category]'" It makes sense to group
## these categories and see if there is a common means of analysis for each.
not_full.sort_values(by="Count")

,Name,Count
57,CIGAGLST,42412
56,MRJAGLST,48234
8,MRDAYPYR,48481
62,ANLAGLST,52908
23,PRDAYPYR,53179
54,MTHAYTOT,53665
67,NMERTMT2,53705
59,HALAGLST,53949
63,TRNAGLST,54068
61,INHAGLST,54158


In [30]:
not_full[not_full.str.contains('LST')]

AttributeError: 'DataFrame' object has no attribute 'str'

<p><b>Next Step:</b> There are 33 remaining categories (35 including bmi and wheight) to be converted to numeric data. Those 33 columns 

In [ ]:
mod_v2['CIGAGE'].value_counts()

In [ ]:
mod_pfd[(mod_pfd == 985).any(axis=1)]

In [ ]:
plt.scatter(pfd['CIGTRY'],pfd['IRMJAGE'])
u = np.linspace(0,1000,2000)
k = pfd['CIGTRY'].corr(pfd['IRMJAGE'])*u
plt.plot(u,k)
plt.show()

In [ ]:
pfd['CIGTRY'].corr(pfd['IRMJAGE'])